In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib
import os

# to disable all debugging logs
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Extracting only potato dataset

In [ ]:
dataset_dir = '/content/drive/MyDrive/DATA  !/archive/PlantVillage'

In [ ]:
#Global Initialization Of some important variables

Image_Size = 256
Batch_Size = 32
Channels = 3
Epochs = 50

In [ ]:

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir, batch_size = Batch_Size, image_size = (Image_Size, Image_Size), shuffle = True) 

In [ ]:

class_name = dataset.class_names
class_name

In [ ]:
len(dataset)

In [ ]:

print(dataset)

In [ ]:
# PLotting Image
plt.figure(figsize = (20,20))


for image, label in dataset.take(1):
    for i in range (12):
        plt.subplot(3, 4, i+1) 
        #converting all data of image into numpy and than to intiger type as they were in float.
        plt.imshow(image[i].numpy().astype('uint8'))
  
        plt.title(class_name[label[i]])
        
        plt.axis('off')
        

# Splitting Dataset

In [ ]:
# Function for Splitting the data

def split_dataset(ds, train_split=0.9, val_split=0.1, test_split=0, shuffle=True, shuffle_size=1000):
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed = 10)
        
    ds_size = len(ds)
    train_size =int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).take(val_size)
    
    return train_ds, val_ds, test_ds

In [ ]:
train_data, val_data, test_data = split_dataset(dataset)

In [ ]:
print("Size of Data is: {0} \nBatch size of Training Data is:{1}\nBatch size of Val data is:{2}\nBatch size of Test Dast Is:{3} " .format(len(dataset), len(train_data), len(val_data), len(test_data)))

In [ ]:
# caching,shuffle and prefetching the data

train_ds = train_data.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
val_da = val_data.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds = test_data.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)

# Pre-processing 

In [ ]:
#Rescaling and Resizing

resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(Image_Size, Image_Size),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [ ]:
# Data augmentation  

data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip(mode = "horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(factor = 0.2)
])

# Model Building

In [ ]:

input_shape = (Batch_Size, Image_Size, Image_Size, Channels)
model = models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', input_shape = input_shape),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64, (3,3), activation = 'relu'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64, (3,3), activation = 'relu'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64, (3,3), activation = 'relu'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64, (3,3), activation = 'relu'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64, (3,3), activation = 'relu'),
    layers.MaxPool2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(3, activation = 'softmax'),

])


model.build(input_shape = input_shape)

In [ ]:
model.summary()

In [ ]:
# Optimizing the model 'SparseCategoricalCrossentropy'=>as there are many categorical classes of data
model.compile(
    optimizer = 'adam', 
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False), 
    metrics = ['accuracy'])

In [ ]:
#Fit the model with training data and also pass validation data
history = model.fit(
train_data, epochs = 100, batch_size = Batch_Size, verbose = 1, validation_data = val_data)

In [ ]:
# SAVE THE MODEL !!!!!!!!!!!!!!!

In [ ]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/models11/potato_model')


In [ ]:
predictions[0]

In [ ]:
history.history.keys()

In [ ]:
# Getting the model history to analyse

train_loss = history.history['loss']
train_acc = history.history['accuracy']

val_loss = history.history['val_loss']
val_acc = history.history['val_accuracy']

In [ ]:
#graphs for accuracy and loss of training and validation data

plt.figure(figsize = (20,20))
plt.subplot(2,3,1)
plt.plot(range(Epochs), train_acc, label = 'Training Accuracy')
plt.plot(range(Epochs), val_acc, label = 'Validation Accuracy')
plt.legend(loc = 'lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2,3,2)
plt.plot(range(Epochs), train_loss, label = 'Training_Loss')
plt.plot(range(Epochs), val_loss, label = 'Validation_Loss')
plt.legend(loc = 'upper right')
plt.title('Training and Validation Loss')

In [ ]:
yyy = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/DATA  !/test/', batch_size = Batch_Size, image_size = (Image_Size, Image_Size), shuffle = False) 

Found 224 files belonging to 1 classes.


In [ ]:
len(batch_list), len(yyy.file_paths)


(224, 224)

In [ ]:
import pandas as pd

In [ ]:
result  = pd.DataFrame(columns=['image','Classification target'])
result

,image,Classification target


In [ ]:
'''
batch_list = []
for batch_image, batch_label in yyy.take(-1):
        for image in batch_image:
          batch_list.append(image)
'''
for label ,img in zip (yyy.file_paths , batch_list):
        filename = label.split('/')[7]
        print(filename)
        '''
        img = tf.expand_dims(img, 0)


    
        batch_prediction = new_model.predict(img)
        predicted_class = class_name[np.argmax(batch_prediction)]

        predicted_class = predicted_class.replace("_", " ")
        if("healthy" in predicted_class ):
           predicted_class = predicted_class.replace("_", " ")
           predicted_class = predicted_class.split(' ')[3]
        else:  
          predicted_class = predicted_class.split(" ")[3]+ " " +predicted_class.split(" ")[4] 
          predicted_class = predicted_class.lower()
        result = result.append({'image': filename, 'Classification target': predicted_class},ignore_index = True)
        print('image = ',filename,' Classification target = ',predicted_class )
'''



01126e2630b15e7e3a0a26869d47ed477e0b7f6c.jpeg
01a83bd0e39561fc090a2dd8f090e15866ddff7d.jpeg
024f92e0e4991f95835d994e7cdc68545fa643c5.jpeg
0571a2fd5825c2b5da83b96bec5da285cbab79b8.jpeg
06b15764355d627b366c4ff5dae2ecd89a9d79fc.jpeg
093286dabe304e991721c02460189fa24c7ac082.jpeg
096345f3166ab13786e5b17bd2480281ef2cf445.jpeg
09eba1cc5442995bd7e6cd16737ca69978348ea9.jpeg
0a10fb71309ca35c7267e958e843d4a0ea3af9ef.jpeg
0b2b8bab509a1fd2d4b4f2b04814ba3811c42fd4.jpeg
0ba04a2d600d193eccab4859524aa91cef705480.jpeg
0c7a3e0a076fbb7b3d6df08d9c3b2bc7ca3f4d02.jpeg
0cff951eef5f8eca43af4fbc3626aa5a760cc90d.jpeg
0e44c9126fe3aa3be505bf18e8822139167990b9.jpeg
0fb9564fd87ee121a6a4a3febf12abce826ed919.jpeg
10e864823a8a984306f178a2b55938354e31cb63.jpeg
118790f58e75aeccebca4e09c2f44b37bb0f6003.jpeg
1384c53475966a0ea72c433fa54c0b2de113a28c.jpeg
17bb9fe264547be2629638ef40b3139132e3a21a.jpeg
1bd4a2b8f6193416bda4a4ffd495afa092cad8e6.jpeg
1e91ea4eb53765d65b88122c11cb3de610313182.jpeg
1edc07ace88530030356313b89e990840d

In [ ]:
result.to_csv("result.csv",index=False)


In [ ]:
print(batch_list[0])

tf.Tensor(
[[[203.22363  194.50488  175.22363 ]
  [205.66602  189.47607  176.9917  ]
  [156.81348  134.87305  125.65723 ]
  ...
  [ 63.06543  107.06543   55.609375]
  [ 87.98193  131.16455   70.38818 ]
  [116.03174  155.07422   89.04053 ]]

 [[126.52002  133.23877   96.75928 ]
  [201.38867  197.71582  168.05225 ]
  [200.03906  181.38281  159.44531 ]
  ...
  [ 90.91504  136.96533   76.04346 ]
  [116.20654  164.84717  100.74512 ]
  [ 81.083984 129.64648   60.802734]]

 [[200.24756  184.51318  161.51318 ]
  [192.96826  174.44043  156.02393 ]
  [201.31445  179.29785  164.64209 ]
  ...
  [ 49.331543  95.862305  22.128418]
  [ 45.84961   99.325195  12.875488]
  [ 55.20996  110.90381   15.825684]]

 ...

 [[196.09033  171.09033  151.09033 ]
  [199.28809  174.28809  154.28809 ]
  [197.71826  172.71826  152.71826 ]
  ...
  [212.7627   187.7627   167.7627  ]
  [203.51074  178.51074  158.51074 ]
  [199.0376   174.0376   154.0376  ]]

 [[188.27637  164.25342  143.20898 ]
  [204.125    179.42822  1

In [ ]:
result = pd.DataFrame()

In [ ]:
result.to_csv("RESULT.csv",index=False,)


In [ ]:
result

,image,Classification target
